# 03 - V-Dem Democracy Analysis

This notebook explores the V-Dem (Varieties of Democracy) database and correlates democracy indicators with human rights defenders data.

## V-Dem Dataset Overview

The V-Dem dataset contains:
- **27,913 observations** (country-year format)
- **4,607 variables** covering democracy indicators
- **Time coverage:** 1789-2024
- **Geographic coverage:** 200+ countries/territories

## Key Democracy Indices

- **Electoral Democracy** (v2x_polyarchy): Free and fair elections
- **Liberal Democracy** (v2x_libdem): Electoral democracy + individual liberties
- **Participatory Democracy** (v2x_partipdem): Active citizen participation
- **Deliberative Democracy** (v2x_delibdem): Reasoned public debate
- **Egalitarian Democracy** (v2x_egaldem): Equal access to political power

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import pearsonr

plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (14, 8)
plt.rcParams['font.size'] = 12

## 1. Load V-Dem Data

In [ ]:
vdem = pd.read_csv('../data/V-Dem-CY-FullOthers-v15_csv/V-Dem-CY-Full+Others-v15.csv', 
                   low_memory=False)

print(f"Dataset: {len(vdem):,} rows x {len(vdem.columns):,} columns")
print(f"Year range: {vdem['year'].min()} - {vdem['year'].max()}")
print(f"Countries: {vdem['country_name'].nunique()}")

## 2. Global Democracy Trends

In [ ]:
# Calculate global averages by year
global_trends = vdem.groupby('year').agg({
    'v2x_polyarchy': 'mean',
    'v2x_libdem': 'mean',
    'v2x_egaldem': 'mean',
    'v2xcl_rol': 'mean'
}).reset_index()

global_trends = global_trends[global_trends['year'] >= 1900]

# Plot
fig, ax = plt.subplots(figsize=(14, 8))

indicators = [
    ('Electoral Democracy', 'v2x_polyarchy', '#2E86AB'),
    ('Liberal Democracy', 'v2x_libdem', '#A23B72'),
    ('Egalitarian Democracy', 'v2x_egaldem', '#F18F01'),
    ('Rule of Law', 'v2xcl_rol', '#6A994E')
]

for name, col, color in indicators:
    ax.plot(global_trends['year'], global_trends[col],
            label=name, linewidth=3, color=color, alpha=0.85)

ax.set_xlabel('Year', fontsize=14)
ax.set_ylabel('Average Index (0-1)', fontsize=14)
ax.set_title('Global Average Democracy Indicators (1900-2024)', fontsize=16)
ax.legend(fontsize=12, loc='lower right')
ax.grid(True, alpha=0.3)

# Historical events
ax.axvspan(1914, 1918, alpha=0.1, color='red')
ax.axvspan(1939, 1945, alpha=0.1, color='red')
ax.axvspan(1989, 1991, alpha=0.1, color='green')

plt.tight_layout()
plt.savefig('../figures/vdem_global_trends.png', dpi=150, bbox_inches='tight')
plt.show()

## 3. Country Comparison

In [ ]:
countries = ['United States of America', 'France', 'Germany', 'China', 'India', 'Brazil']
country_data = vdem[(vdem['year'] >= 1900) & (vdem['country_name'].isin(countries))]

fig, ax = plt.subplots(figsize=(14, 8))

colors = ['#2E86AB', '#A23B72', '#F18F01', '#C73E1D', '#6A994E', '#8B5CF6']

for i, country in enumerate(countries):
    data = country_data[country_data['country_name'] == country]
    ax.plot(data['year'], data['v2x_polyarchy'], 
            label=country, linewidth=2.5, color=colors[i], alpha=0.85)

ax.set_xlabel('Year', fontsize=14)
ax.set_ylabel('Electoral Democracy Index', fontsize=14)
ax.set_title('Electoral Democracy Index by Country (1900-2024)', fontsize=16)
ax.legend(fontsize=11, loc='upper left')
ax.set_ylim(-0.05, 1.05)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('../figures/vdem_country_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

## 4. Correlation with Human Rights Defenders

Compare V-Dem democracy trends with our Chao1 estimates of human rights defenders.

In [ ]:
# Load Chao1 results
chao_df = pd.read_csv('../data/chao1_results_world.csv')

# Prepare democracy data by decade
vdem['decade'] = (vdem['year'] // 10) * 10
democracy_decade = vdem.groupby('decade')['v2x_libdem'].mean().reset_index()
democracy_decade.columns = ['decade', 'democracy']

# Merge datasets
merged = pd.merge(democracy_decade, chao_df[['decade', 'chao1_estimate']], on='decade')
merged = merged[merged['decade'] <= 2010]

# Normalize both series (0-1 scale)
merged['democracy_norm'] = (merged['democracy'] - merged['democracy'].min()) / \
                           (merged['democracy'].max() - merged['democracy'].min())
merged['chao1_norm'] = (merged['chao1_estimate'] - merged['chao1_estimate'].min()) / \
                       (merged['chao1_estimate'].max() - merged['chao1_estimate'].min())

# Calculate correlation
corr, pval = pearsonr(merged['democracy_norm'], merged['chao1_norm'])
print(f"Pearson correlation: {corr:.3f} (p-value: {pval:.4f})")

In [ ]:
# Plot correlation
fig, ax = plt.subplots(figsize=(14, 8))

ax.plot(merged['decade'], merged['democracy_norm'], 
        marker='o', linewidth=3, markersize=10,
        label='Liberal Democracy (V-Dem)', color='#1976D2')
ax.plot(merged['decade'], merged['chao1_norm'], 
        marker='s', linewidth=3, markersize=10,
        label='Human Rights Defenders (Chao1)', color='#D32F2F')

ax.set_xlabel('Decade', fontsize=14)
ax.set_ylabel('Normalized Score (0-1)', fontsize=14)
ax.set_title(f'Democracy vs Human Rights Defenders (r = {corr:.3f})', fontsize=16)
ax.legend(fontsize=12)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('../figures/democracy_hrd_correlation.png', dpi=150, bbox_inches='tight')
plt.show()

## 5. Current Democracy Rankings

In [ ]:
latest_year = vdem['year'].max()
latest = vdem[vdem['year'] == latest_year][['country_name', 'v2x_polyarchy']].copy()

print(f"Top 10 Most Democratic Countries ({latest_year}):")
print("=" * 50)
print(latest.nlargest(10, 'v2x_polyarchy').to_string(index=False))

print(f"\nBottom 10 Least Democratic Countries ({latest_year}):")
print("=" * 50)
print(latest.nsmallest(10, 'v2x_polyarchy').to_string(index=False))

## 6. Key Findings

1. **Global democracy trends** show significant fluctuations, with drops during world wars and recovery afterward
2. **Strong correlation** between democracy levels and the emergence of human rights defenders
3. **Regional variations** are substantial, with Western democracies consistently higher than authoritarian states